# Base de datos diario -ICC

**Objetivo**
Con este notebook se reproduce el excel que ICC maneja como insumo para sus calculos, reporte y demas.

In [ ]:
# check donde estamos trabajando
#pwd

In [ ]:
# libreria para movernos entre diferentes rutas
#import os

#os.chdir('../../')

In [ ]:
#pip install -U scikit-learn

In [5]:
# Libreías generales
import numpy as np
import pandas as pd
import math
import datetime
from dateutil.relativedelta import relativedelta
import glob
import datetime
import sklearn

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
import matplotlib.font_manager
from matplotlib import style
style.use('ggplot') or plt.style.use('ggplot')
import seaborn as sns



# Preprocesado y modelado
# ==============================================================================
from sklearn.cluster import DBSCAN, KMeans
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors


import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

In [6]:
#comando para visualizar todas las columnas del df
pd.pandas.set_option('display.max_columns', None)

# features load
los datos aqui cargados sufireron una limpieza en el notebook 

In [ ]:
# get data file names
path =r'/home/diana/Escritorio/CIAT/ICC/historicos_horarios_excel/'
filenames = glob.glob(path + "/*.xlsx")
filenames

In [ ]:
%%time
dfs = pd.DataFrame()
li=[]

for csv in filenames:
    df = pd.read_excel(csv, index_col=None)
    li.append(df)

dfs = pd.concat(li, axis=0, ignore_index=True)
dfs.head()

In [ ]:
dfs.drop('Unnamed: 0',axis=1, inplace=True)

In [ ]:
dfs

pd.read_parquet('icc_stations_all.parquet')


In [7]:
dfs = pd.read_parquet('icc_stations_all.parquet')
dfs.head(20)

FileNotFoundError: [Errno 2] No such file or directory: 'icc_stations_all.parquet'

# Agregacion diaria

In [ ]:
dfs.info()

In [ ]:
def segmentacion(df, col_fecha):
    """
    función que segmenta la fecha en dia, mes, año y semana.
    
    Arguments:
        df: dataframe 
        col_fecha: columna que contiene la informacion de la fecha
        
    Returns:
        df: dataframe de entrada con 5 nuevas columnas que informan sobre laç
        fecha dd/mm/aa, semana, dia, mes y año
    """
    df['date'] = pd.to_datetime(df[col_fecha]).dt.date
    df['semana'] = pd.to_datetime(df[col_fecha]).dt.week
    df['dia'] = pd.to_datetime(df[col_fecha]).dt.day
    df['mes'] = pd.to_datetime(df[col_fecha]).dt.month
    df['año'] = pd.to_datetime(df[col_fecha]).dt.year
    return df

In [ ]:
#aqui se llama la funcion y se pone los parametros
segmentacion(dfs, 'fecha')

# Agregaciones diarias
con la siguiente linea se construyen las columnas agregadas que tenia carlos en el excel

In [ ]:
df_diario = dfs.groupby(['estacion', 'date','semana','dia','mes','año']).agg(
    temperatura_min_diaria=('temperatura', 'min'),
    temperatura_max_diaria=('temperatura', 'max'),
    temperatura_promedio_diaria=('temperatura', 'mean'),
    radiacion_diaria_acumulada=('radiacion', 'sum'),
    radiacion_diaria_promedio = ('radiacion','mean'),
    humedad_relativa_min_diaria = ('humedad_relativa', 'min'),
    humedad_relativa_max_diaria = ('humedad_relativa', 'max'),
    humedad_relativa_media_diaria = ('humedad_relativa', 'mean'),
    lluvia_diaria = ('precipitacion', 'sum'),
    velocidad_viento_media_diario = ('velocidad_viento', 'mean'),
    velocidad_viento_max_diaria = ('velocidad_viento', 'max')
).reset_index()

In [ ]:
df_diario['amplitud_termica'] = df_diario.temperatura_max_diaria - df_diario.temperatura_min_diaria

# constantes para algunos calculos de variables
las tablas que se consultan fueron sacadas del excel compartido por ICC. se esta adelantando la revision bibliografica para generalizar estas tablas a otras estaciones.

In [ ]:
"""
constantes de radiacion extraterreste y numero de dias despejado
"""

ra_n_ctes = pd.read_excel('tabla_Ra_N.xlsx').iloc[:-1]
ra_n_ctes.pop("Unnamed: 22")

ctes_radiacion = pd.read_excel('constantes_radiacion.xlsx')
ctes_radiacion.head()

In [ ]:
ctes_location = pd.read_excel('estra_location.xlsx')
ctes_location['latitud_rad'] = ctes_location['latitud'].apply(lambda row: math.radians(row))

In [ ]:
def extraer_info(df, filtros_str, name_feature, name_col):
    """
    Como la tabla tiene la informacion de varias constantes
    de interes como por ejemplo: radiacion general y radiacion en dia p,
    se hace un filtro para separalas. como hacemos el filtro nos queda en las columnas los meses y en las filas con el dia.
    es necesario quitar  el sufijo lpara solo quedarnos con el numero de meses.

    la matriz anterior se re diseña para que el mes ya no quede como columna sino como filas
    y los valores de la radiacion no quede en la matriz sino en una columna
    
    
    Arguments:
        df: dataframe 
        col_fecha: columna que contiene la informacion de la fecha
        
    Returns:
        df: dataframe de entrada con 5 nuevas columnas que informan sobre laç
        fe

    """
    matriz=df.filter(regex=filtros_str)
    matriz.columns = matriz.columns.str.replace("_" + name_feature, "")
    df_feature = matriz.melt(id_vars=["dia"], 
        var_name="mes", 
        value_name=name_col)
    df_feature.mes = df_feature.mes.astype('int')
    return df_feature

In [ ]:
df_n = extraer_info(ra_n_ctes, 'dia|N', 'N', 'N Daylight hours')
df_ra = extraer_info(ra_n_ctes, 'dia|Ra', 'Ra', 'Ra')

In [ ]:
df_rg = extraer_info(ctes_radiacion, 'rgl|dia', 'rgl', 'radiacion_global')
df_rdp = extraer_info(ctes_radiacion, 'rdp|dia', 'rdp', 'radiacion_dia_despejado')

## Join all dfs

In [ ]:
"""
se unen los 3 dataframes donde las columnas comunes o las llaves son dia y mes y se 
reescribe el df_diario

"""
df_diario  = df_diario.merge(df_rg, how='inner', on=['dia', 'mes']).merge(df_rdp, how='inner', on=['dia', 'mes'])
df_diario = df_diario.merge(df_n, how='inner', on=['dia', 'mes']).merge(df_ra, how='inner', on=['dia', 'mes'])
##
df_diario = df_diario.merge(ctes_location, how= 'inner', on=['estacion'])

## Calculo de variables secundarias (las variables que requieren calculos)

In [ ]:
def secundary_variables(df):
    df['radiacion_media_estimada Heargreaves'] = 0.16 * (
        np.sqrt(df.amplitud_termica)
    ) * df.radiacion_global
    
    df['Rg'] = 0.00089681 * df.radiacion_diaria_acumulada
    
    df['n']=(
        -0.32 +1.61*(df.Rg / df.Ra)
    )*df['N Daylight hours']
    
    ## balance hidrico
    
    df['dia_juliano'] = df['date'].apply(
        lambda row: (row -datetime.date(row.year,1,1)).days +1
    )
    
    df['dr'] = 1 + (0.033 * np.cos(((2*3.1416)/365)*df['dia_juliano']))
    
    df['declinacion'] = 0.409  * np.sin(
        (((2*3.1416)/365)*df['dia_juliano'])-1.39
    )
    
    df['ang_horario_puesta_sol'] = np.arccos(
        -(np.tan(df.latitud_rad) * np.tan(df.declinacion))
    )
    
    df['heliofania'] = (24/3.1416) * df['ang_horario_puesta_sol']
    
    df['ra_MJm-2day-1'] = ((24*60)/3.1416) * (0.082*df['dr']) * (
        (
            (df.ang_horario_puesta_sol * np.sin(df.latitud_rad))*(np.sin(df.declinacion))
        )+(
            np.cos(df.latitud_rad)*np.cos(df.declinacion)*np.sin(df.ang_horario_puesta_sol)
        )
    )
    
    df['radiacion_solar_piranometro'] = 0.0864 * df.radiacion_diaria_promedio
    
    df['constante_psicometrica'] =(
        (
            pow(((293-(0.0065 * df.altitud))/293), 5.26)
        )*101.3
    )*0.000665
    
    df['pendiente_curva_presion_satu_vapor'] = (
        4089 * (
            (0.6108 * (
                np.exp((17.27*df.temperatura_promedio_diaria)/(df.temperatura_promedio_diaria+237.3))
            )) / pow(
                (df_diario.temperatura_promedio_diaria+273.3),
                2)
        )
    )
    
    df['presion_saturacion_vapor'] = (
        (
            0.6108 * np.exp(
                (17.27*df.temperatura_min_diaria)/(df.temperatura_min_diaria +237.3)
            )
        )+(
            0.6108*np.exp(
                (17.27*df.temperatura_max_diaria)/(df.temperatura_max_diaria+237.3)
            )
        )
    )/2
    
    
    df['presion_real_vapor'] = (
        (
            0.6108*np.exp(
                (17.27*df.temperatura_max_diaria)/(df.temperatura_max_diaria +237.3)
            )
        ) * (df.humedad_relativa_min_diaria /100) + (
            0.6108*np.exp(
                (17.27*df.temperatura_min_diaria) / (df.temperatura_min_diaria+273.3)
            )
        ) * (df.humedad_relativa_max_diaria/100)
    )/2
    
    df['deficit_presion_vapor'] = df['presion_saturacion_vapor'] - df['presion_real_vapor']
    
    df['rns'] = df.radiacion_solar_piranometro * 0.77
    
    df['rnl']= (
        (
            (
                (
                    0.000000004903*(pow((df.temperatura_min_diaria+273.16),4))
                ) + (
                    (0.000000004903 * (pow(
                        (df.temperatura_max_diaria+273.16)
                        ,4)))
                )
            ) / 2
        ) * (
            0.34 -(0.14 *np.sqrt(df.presion_real_vapor))
        ) * (
            1.35 * (
                df.radiacion_solar_piranometro /((0.75+2*(df.altitud/100000))* df['ra_MJm-2day-1'])
            ) -0.35
        )
    )
    
    df['rn'] = df['rns'] - df['rnl']
    
    df['velocidad_viento_altura_standar'] = (
        (4.87)/((np.log((67.8*10)-5.42)))
    )*(
        (df.velocidad_viento_media_diario * 1000)/3600
    )
    
    df['FAO-Penman-Montieth'] = (
        (0.408 * df.pendiente_curva_presion_satu_vapor * df.rn) + (
            df.constante_psicometrica * (
                900/(df.temperatura_promedio_diaria + 273)
            )* df.velocidad_viento_altura_standar * df.deficit_presion_vapor
        )
    ) / (
        df.pendiente_curva_presion_satu_vapor + (
            df.constante_psicometrica + (
                1+(0.34 + df.velocidad_viento_altura_standar)
            )
        )
    )
    
    df['deficit']= df.lluvia_diaria - df['FAO-Penman-Montieth']
    return df

In [ ]:
secundary_variables(df_diario)

In [ ]:
df_diario.to_parquet('diario_complete.parquet')

# el raound es para el redondeo
df_diario['radiacion_media_estimada Heargreaves'] = round(
    0.16 * (np.sqrt(df_diario.amplitud_termica)) * df_diario.radiacion_global,
    1
)


df_diario['Rg']=0.00089681*df_diario.radiacion_diaria_acumulada

df_diario['n']=round(
    (-0.32 +1.61*(df_diario['Rg']/df_diario['Ra']))*df_diario['N Daylight hours'],
    1
)

In [ ]:


secundary_variables(df_diario)

In [ ]:
df_diario.head()

In [ ]:
def

In [ ]:
df_diario.to_parquet('df_diario.parquet')

# Balance hidrico

In [ ]:
df_diario = pd.read_parquet('df_diario.parquet')

In [ ]:
df_diario.head()

df_diario['dia_juliano'] = df_diario['date'].apply(
    lambda row: (row -datetime.date(row.year,1,1)).days +1
)

df_diario['dr'] = 1+(0.033*np.cos(((2*3.1416)/365)*df_diario['dia_juliano']))

df_diario['declinacion'] = 0.409*np.sin((((2*3.1416)/365)*df_diario['dia_juliano'])-1.39)

# lectura sobre altitud y latitud
A partir de aqui solo se quedan las 27 estaciones que analizan en ICC. antes habian 32

ctes_location = pd.read_excel('estra_location.xlsx')
ctes_location

ctes_location['latitud_rad'] = ctes_location['latitud'].apply(lambda row: math.radians(row))

df_diario = df_diario.merge(ctes_location, how= 'inner', on=['estacion'])

df_diario['ang_horario_puesta_sol'] = np.arccos(
    -(
        np.tan(df_diario.latitud_rad) * np.tan(df_diario.declinacion)
    )
)

df_diario['heliofania'] =(24/3.1416)*df_diario['ang_horario_puesta_sol']

df_diario['ra_MJm-2day-1'] = (
    (24*60)/3.1416
) *(
    0.082*df_diario['dr']
)*(
    (
        (
            df_diario.ang_horario_puesta_sol * np.sin(df_diario.latitud_rad)
        )*(
            np.sin(df_diario.declinacion)
        )
    )+(
        np.cos(df_diario.latitud_rad)*np.cos(df_diario.declinacion)*np.sin(df_diario.ang_horario_puesta_sol)
    )
)


df_diario['radiacion_solar_piranometro'] = 0.0864*df_diario.radiacion_diaria_promedio

df_diario['constante_psicometrica'] =(
    (
        pow(
            (
                (293-(0.0065*df_diario.altitud))/293
            )
            , 5.26
        )
    )*101.3
)*0.000665

df_diario['pendiente_curva_presion_satu_vapor'] =(
    4089*(
        (
            0.6108*(np.exp((17.27*df_diario.temperatura_promedio_diaria)/(df_diario.temperatura_promedio_diaria+237.3))))/pow((df_diario.temperatura_promedio_diaria+273.3), 2)))

df_diario['presion_saturacion_vapor'] =((0.6108*np.exp((17.27*df_diario.temperatura_min_diaria)/(df_diario.temperatura_min_diaria +237.3)))+(0.6108*np.exp((17.27*df_diario.temperatura_max_diaria)/(df_diario.temperatura_max_diaria+237.3))))/2

df_diario['presion_real_vapor'] = (
    (
        0.6108*np.exp(
            (
                17.27*df_diario.temperatura_max_diaria
            )/(
                df_diario.temperatura_max_diaria +237.3
            )
        )
    ) * (
        df_diario.humedad_relativa_min_diaria /100
    ) + (
        0.6108*np.exp(
            (
                17.27*df_diario.temperatura_min_diaria
            )/ (
                df_diario.temperatura_min_diaria+273.3
            )
        )
    ) * (
        df_diario.humedad_relativa_max_diaria/100
    )
)/2

df_diario['deficit_presion_vapor'] = df_diario['presion_saturacion_vapor'] - df_diario['presion_real_vapor']

df_diario['rns']=df_diario.radiacion_solar_piranometro * 0.77

df_diario['rnl']= (
    (
        (
            (
                0.000000004903*(
                    pow(
                        (df_diario.temperatura_min_diaria+273.16),
                        4
                    )
                )
            ) + (
                (
                    0.000000004903 * (
                        pow(
                            (df_diario.temperatura_max_diaria+273.16),
                            4
                        )
                    )
                )
            )
        ) / 2
    ) * (
        0.34 -(
            0.14 *np.sqrt(df_diario.presion_real_vapor)
        )
    ) * (
        1.35 * (
            df_diario.radiacion_solar_piranometro /(
                (
                    0.75+2*(df_diario.altitud/100000)
                )* df_diario['ra_MJm-2day-1']
            )
        ) -0.35
    )
)
            

df_diario['rn']=df_diario['rns']-df_diario['rnl']

df_diario['velocidad_viento_altura_standar'] = (
    (4.87)/(
        (np.log((67.8*10)-5.42))
    )
)*(
    (df_diario.velocidad_viento_media_diario*1000)/3600
)

df_diario['FAO-Penman-Montieth'] = (
    (
        0.408*df_diario.pendiente_curva_presion_satu_vapor*df_diario.rn
    )+(
        df_diario.constante_psicometrica*(
            900/(df_diario.temperatura_promedio_diaria + 273)
        )*df_diario.velocidad_viento_altura_standar * df_diario.deficit_presion_vapor
    )
) / (
    df_diario.pendiente_curva_presion_satu_vapor + (
        df_diario.constante_psicometrica + (
            1+(0.34 + df_diario.velocidad_viento_altura_standar)
        )
    )
)
    

In [ ]:
df_diario['deficit']= df_diario.lluvia_diaria-df_diario['FAO-Penman-Montieth']